In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install -U textblob
!python -m textblob.download_corpora

Requirement already up-to-date: textblob in /usr/local/lib/python3.6/dist-packages (0.15.3)
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package conll2000 to /root/nltk_data...
[nltk_data]   Package conll2000 is already up-to-date!
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
Finished.


In [ ]:
!pip install textattack

     |████████████████████████████████| 327kB 2.0MB/s 
     |████████████████████████████████| 235kB 6.2MB/s 
     |████████████████████████████████| 1.7MB 8.1MB/s 
     |████████████████████████████████| 153kB 16.9MB/s 
     |████████████████████████████████| 1.1MB 13.9MB/s 
     |████████████████████████████████| 686kB 15.6MB/s 
     |████████████████████████████████| 778kB 22.7MB/s 
     |████████████████████████████████| 3.0MB 21.0MB/s 
     |████████████████████████████████| 102kB 9.4MB/s 
     |████████████████████████████████| 61kB 6.9MB/s 
     |████████████████████████████████| 61kB 7.0MB/s 
     |████████████████████████████████| 317kB 38.5MB/s 
     |████████████████████████████████| 245kB 45.4MB/s 
     |████████████████████████████████| 102kB 9.9MB/s 
     |████████████████████████████████| 102kB 10.1MB/s 
     |████████████████████████████████| 163kB 38.9MB/s 
     |████████████████████████████████| 122kB 38.3MB/s 
     |████████████████████████████████| 245kB 47.6MB/s 
 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import spacy
import time
from textblob import TextBlob

# install googletrans using pip
!pip install googletrans

# import the library
import googletrans
from googletrans import Translator

In [ ]:
english = pd.read_csv("/content/drive/My Drive/Colab Notebooks/reviews_moderation/reviews_translated.csv") 
#english.sample(5)
english.dropna()

,Review,Approved,TotalScore,PunctualityScore,TranslatedReview
0,"Καθησυχαστικη, έμπειρη και αποτέλεσμάτικη",1,10,10,"Reassuring, experienced and effective"
1,Υπέροχος άνθρωπος σε κάνει να την εμπιστευτείς...,1,10,10,A wonderful person makes you trust her and be ...
2,"O γιατρός είναι εξαιρετικος, γνωστής του αντικ...",1,10,10,"The doctor is excellent, familiar with the sub..."
3,Μου ενέπνευσε εμπιστοσύνη και είπα πράγματα πο...,1,10,10,It gave me confidence and I said things I have...
4,Πολύ καλός γιατρός Τον συστήνω ανεπιφύλακτα. ...,1,10,10,Very good doctor I highly recommend him. Absol...
...,...,...,...,...,...
127207,Τον συστηνω για οσους τον εχουν αναγκη.,1,10,10,I recommend it for those who need it.
127208,πηγα στο ραντεβου αλλα δεν ηταν ενημερωμενοι γ...,0,0,2,I went to the appointment but they were not in...
127209,Ειμαι πολύ ικανοποιημένη με τον εν λογω γιατρο...,1,10,10,I am very satisfied with this doctor and his s...
127210,άψογη συμπεριφορά και υψηλός επαγγελματισμός,1,10,10,impeccable behavior and high professionalism


In [ ]:
english = english[['TranslatedReview','Approved']]

In [ ]:
english_not_approved = english[english['Approved']==0]
english_not_approved

,TranslatedReview,Approved
492,xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx...,0
506,..............................,0
514,👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍,0
535,+11111111111111111!!!!!!!!!!!!,0
578,..............................,0
...,...,...
127178,"Very helpful, talkative, communicative, he exp...",0
127182,EIXA CONFIRMED APPOINTMENT FROM THE SITE FOR M...,0
127192,He took us a 20 euro visit to issue a basketba...,0
127208,I went to the appointment but they were not in...,0


Augment with text attack

In [ ]:
train_aug = english_not_approved.TranslatedReview.copy()
from textattack.augmentation import WordNetAugmenter
aug = WordNetAugmenter()

train_aug = train_aug.apply(lambda x: str(aug.augment(str(x))))

In [ ]:
train_aug

492       ['xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx...
506                      ['..............................']
514               ['👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍']
535                      ['+11111111111111111!!!!!!!!!!!!']
578                      ['..............................']
                                ...                        
127178    ['identical helpful, talkative, communicative,...
127182    ['EIXA CONFIRMED APPOINTMENT FROM THE SITE FOR...
127192    ["helium hire us a 20 euro visit to issuance a...
127208    ['I went to the appointment but they were not ...
127211               ['Gentle, approachable, instructive.']
Name: TranslatedReview, Length: 4111, dtype: object

In [ ]:
train_aug = train_aug.str.replace("[", "").str.replace("]","").str.replace("'","")

In [ ]:
train_aug[30:60]

3350    A lot of weft selfsame little lesson A little ...
3357    I have the best printing from Dr. Ko Galouzis....
3409    "Unacceptable. I told her that I had anxiety o...
3416    Although he is not our doctor and I travel wit...
3423    very well-chosen with the date, inspires confi...
3494    I went to test my myopia to begin lenses. My d...
3495    "Friendly and caring for the patient. Very con...
3498    Very expensive. He ask me for 80 EUR for 5 loc...
3518    iodine still can not value him because the fir...
3520    The appointment was confirmed by you and becau...
3578    civilised, fully supportive with understanding...
3808        .............................................
3834    "He did not follow the necessary hygiene rules...
3942    I choose this MD because it had very good revi...
3979    It is stated on the doctoranytime situation th...
3993    CONSISTENT AND receive. EXAMINED ME IN DETAIL ...
3994    "Read this first. The website was simply not i...
4097    the do

In [ ]:
english_not_approved.TranslatedReview[30:60]

3350    A lot of weft Very little lesson A little subs...
3357    I have the best impression from Dr. Ko Galouzi...
3409    Unacceptable. I told her that I had anxiety at...
3416    Although he is not our doctor and I went with ...
3423    very happy with the date, inspires confidence,...
3494    I went to test my myopia to get lenses. My doc...
3495    Friendly and caring for the patient. Very conv...
3498    Very expensive. He asked me for 80 EUR for 5 l...
3518    I still can not evaluate him because the first...
3520    The appointment was confirmed by you and becau...
3578    Polite, fully supportive with understanding, a...
3808        .............................................
3834    He did not follow the necessary hygiene rules ...
3942    I chose this doctor because it had very good r...
3979    It is stated on the doctoranytime site that it...
3993    CONSISTENT AND EXPERIENCED. EXAMINED ME IN DET...
3994    Read this first. The website was simply not in...
4097    the do

In [ ]:
train_aug

492       xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx...
506                          ..............................
514                   👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍
535                          +11111111111111111!!!!!!!!!!!!
578                          ..............................
                                ...                        
127178    Very helpful, talkative, communicative, he exp...
127182    EIXA CONFIRMED APPOINTMENT FROM THE SITE FOR M...
127192    "He took us a 20 euro visit to issue a hoops c...
127208    I fail to the appointment but they were not in...
127211                  ennoble, approachable, informative.
Name: TranslatedReview, Length: 4111, dtype: object

#### Adding toxic comments

In [ ]:
toxic = pd.read_csv("/content/drive/My Drive/Colab Notebooks/policy_breaking_reviews.csv", delimiter=';') 
toxic = toxic.dropna(how='all')
toxic = toxic.dropna(axis='columns', how = 'all')
toxic["Approved"] = toxic.Approved.astype(int)
toxic.columns = ['review', 'approved']

In [ ]:
from tqdm import tqdm
start = time.time()
d = []
#batches of 20000
for rev in tqdm(toxic.review[:]):
  try:
    blob_obj = TextBlob(rev)
    translated = str(blob_obj.translate(to="en"))
    time.sleep(0.5)
    #print(blob_obj.translate(to="en"))
    d.append(translated)
    #revs.review[rev] = blob_obj.translate(to="en")
  except Exception as e:
    d.append(rev)
    print(rev)
    print(str(e))
    #time.sleep(1)
end = time.time()
print("it took:", end - start ,"seconds")

100%|██████████| 128/128 [01:18<00:00,  1.63it/s]

it took: 78.6714563369751 seconds


In [ ]:
translated_toxic = pd.DataFrame(d, columns=['review'])
translated_toxic

,review
0,The doctor is a big fool. he did not examine m...
1,"We are talking, the guy took 100 euros and did..."
2,my cell phone is 690000000 take me now
3,ρεεεεεεεεεεεε. what is this. stinky doctor's o...
4,The doctor is a big punch. I're not going to s...
...,...
123,Big guy the guy. He created a problem in my st...
124,Καθίκι !! I have nothing else to say!
125,After the treatment I started to feel dizzy !!...
126,I got a checkered rash all over my body with t...


In [ ]:
toxic_train_aug = translated_toxic.TranslatedReview.copy()
from textattack.augmentation import WordNetAugmenter
aug = WordNetAugmenter()

toxic_train_aug = toxic_train_aug.apply(lambda x: str(aug.augment(str(x))))

In [ ]:
toxic_train_aug = toxic_train_aug.str.replace("[", "").str.replace("]","").str.replace("'","")

In [ ]:
toxic_train_aug

0      The doctor is a big fool. he did not analyse m...
1      We are talking, the hombre took 100 euros and ...
2            my cellphone phone is 690000000 take me now
3      "ρεεεεεεεεεεεε. what is this. stinky doctors p...
4      "The doctor is a big biff. Ire not going to se...
                             ...                        
123    bounteous guy the guy. He created a problem in...
124              Καθίκι !! ane have nothing else to say!
125    after the treatment I started to feel dizzy !!...
126    I got a checkered rash all over my body with t...
127    The guy was the biggest sexist I have ever met...
Name: TranslatedReview, Length: 128, dtype: object

In [ ]:
english_not_approved_augm = pd.DataFrame(train_aug)
english_not_approved_augm

,TranslatedReview
492,xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx...
506,..............................
514,👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍
535,+11111111111111111!!!!!!!!!!!!
578,..............................
...,...
127178,"identical helpful, talkative, communicative, h..."
127182,EIXA CONFIRMED APPOINTMENT FROM THE SITE FOR M...
127192,"""helium hire us a 20 euro visit to issuance a ..."
127208,I went to the appointment but they were not in...


In [ ]:
translated_toxic.columns = ['TranslatedReview']

In [ ]:
result = english_not_approved_augm.append(translated_toxic, ignore_index=True)
result

,TranslatedReview
0,xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx...
1,..............................
2,👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍
3,+11111111111111111!!!!!!!!!!!!
4,..............................
...,...
4234,Big guy the guy. He created a problem in my st...
4235,Καθίκι !! I have nothing else to say!
4236,After the treatment I started to feel dizzy !!...
4237,I got a checkered rash all over my body with t...


append the textattack augmented toxic comments

In [ ]:
result_new = result.append(translated_toxic, ignore_index=True)

In [ ]:
result_new

,TranslatedReview
0,xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx...
1,..............................
2,👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍
3,+11111111111111111!!!!!!!!!!!!
4,..............................
...,...
4362,Big guy the guy. He created a problem in my st...
4363,Καθίκι !! I have nothing else to say!
4364,After the treatment I started to feel dizzy !!...
4365,I got a checkered rash all over my body with t...


export to csv the augmented english set

In [ ]:
result_new

,TranslatedReview
0,xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx...
1,..............................
2,👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍
3,+11111111111111111!!!!!!!!!!!!
4,..............................
...,...
4362,Big guy the guy. He created a problem in my st...
4363,Καθίκι !! I have nothing else to say!
4364,After the treatment I started to feel dizzy !!...
4365,I got a checkered rash all over my body with t...


In [ ]:
result_new.to_csv('/content/drive/My Drive/Colab Notebooks/reviews_moderation_v2/data/English/not_approved_augmented_data_english.csv', header=True, index=False)

In [ ]:
not_approved_aug_english = pd.read_csv("/content/drive/My Drive/Colab Notebooks/reviews_moderation_v2/data/English/not_approved_augmented_data_english.csv", delimiter=';') 

In [ ]:
not_approved_aug_english = not_approved_aug_english[['TranslatedReview','Approved']]

In [ ]:
not_approved_aug_english

,TranslatedReview,Approved
0,xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx...,0
1,..............................,0
2,👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍,0
3,+11111111111111111!!!!!!!!!!!!,0
4,..............................,0
...,...,...
4362,Big guy the guy. He created a problem in my st...,0
4363,Καθίκι !! I have nothing else to say!,0
4364,After the treatment I started to feel dizzy !!...,0
4365,I got a checkered rash all over my body with t...,0


In [ ]:
english

,TranslatedReview,Approved
0,"Reassuring, experienced and effective",1
1,A wonderful person makes you trust her and be ...,1
2,"The doctor is excellent, familiar with the sub...",1
3,It gave me confidence and I said things I have...,1
4,Very good doctor I highly recommend him. Absol...,1
...,...,...
127207,I recommend it for those who need it.,1
127208,I went to the appointment but they were not in...,0
127209,I am very satisfied with this doctor and his s...,1
127210,impeccable behavior and high professionalism,1


In [ ]:
final_translated_reviews = english.append(not_approved_aug_english, ignore_index=True)
final_translated_reviews

,TranslatedReview,Approved
0,"Reassuring, experienced and effective",1
1,A wonderful person makes you trust her and be ...,1
2,"The doctor is excellent, familiar with the sub...",1
3,It gave me confidence and I said things I have...,1
4,Very good doctor I highly recommend him. Absol...,1
...,...,...
131574,Big guy the guy. He created a problem in my st...,0
131575,Καθίκι !! I have nothing else to say!,0
131576,After the treatment I started to feel dizzy !!...,0
131577,I got a checkered rash all over my body with t...,0


In [ ]:
final_translated_reviews.to_csv('/content/drive/My Drive/Colab Notebooks/reviews_moderation_v2/data/English/not_approved_augmented_data_english.csv', header=True, index=False)